## add mvncall multiallelics/1570 to the haplotype genotype array

In [1]:
%run setup.ipynb

### get phased haplotypes for 2L centromere proximal region

In [2]:
# setup haplotype data
callset_phased = phase1_ar31.callset_phased
genotypes_phased = allel.GenotypeDaskArray(callset_phased['2L/calldata/genotype'])
pos_phased = allel.SortedIndex(callset_phased['2L/variants/POS'])

In [5]:
genotypes_phased.shape, pos_phased.shape

((8296600, 773, 2), (8296600,))

In [6]:
#kdr locations
pos_kdr_s = 2422651
pos_kdr_f = 2422652

In [7]:
# define region we're going to analyse
loc_region = pos_phased.locate_range(0, 4000000)
pos_phased_region = pos_phased[loc_region]
pos_phased_region

0,1,2,3,4,...,163958,163959,163960,163961,163962
44688,44691,44732,44736,44756,...,3997372,3997373,3997378,3997381,3997386


In [19]:
# chop genotypes to region, remove colony parents (8 samples) and turn into haplotype array
gen_phased_region = genotypes_phased[loc_region][:, :-8].compute()
gen_phased_region.shape
# don't turn in .to_haplotypes() yet - might be easier to interleave new positions first, then turn whole lot into hap

(163963, 765, 2)

In [21]:
gen_phased_region

<GenotypeArray shape=(163963, 765, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

### grab the mvncalls (in npz format)

In [51]:
#genotypes
callset_nu = np.load('../data/missense_multiallelics_and_1570.mvncall.200.npz')
calldata_nu = callset_nu['calldata']
gen_nu = allel.GenotypeArray(calldata_nu['genotype'])

In [64]:
#positions
pos_nu = allel.SortedIndex(callset_nu['variants']['POS'])
pos_nu

0,1,2
2391228,2400071,2429745


# weave the genotypes and positions into the phase1 haplotype data

### Alistair's weave technique

In [76]:
#concatenate old and new gen/pos arrays
haps_combined = np.concatenate([gen_phased_region, gen_nu], axis=0)
pos_combined = np.concatenate([pos_phased_region, pos_nu], axis=0)

In [77]:
#sort pos indices
idx_sorted =np.argsort(pos_combined)

In [78]:
#use sorted indices to re-order the combined arrays
haps_combined = allel.GenotypeArray(haps_combined[idx_sorted])
haps_combined.shape

(163966, 765, 2)

In [79]:
pos_combined = allel.SortedIndex(pos_combined[idx_sorted])
pos_combined.shape

(163966,)

In [ ]:
#then turn into haplotype array....

### Kaibosely's weave technique

In [80]:
gen_nu

,0,1,2,3,4,...,760,761,762,763,764
0,0/0,2/0,0/0,0/0,0/2,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
2,1/0,0/0,1/1,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0


In [81]:
np.insert(gen_nu, 0, 5, axis=0)

array([[[5, 5],
        [5, 5],
        [5, 5],
        ..., 
        [5, 5],
        [5, 5],
        [5, 5]],

       [[0, 0],
        [2, 0],
        [0, 0],
        ..., 
        [0, 0],
        [0, 0],
        [0, 0]],

       [[0, 0],
        [0, 0],
        [0, 0],
        ..., 
        [0, 0],
        [0, 0],
        [0, 0]],

       [[1, 0],
        [0, 0],
        [1, 1],
        ..., 
        [0, 0],
        [0, 0],
        [0, 0]]], dtype=int8)